In [1]:
import pandas as pd
import numpy as np


# Load data
#pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]

data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp']) 
data['k_pc'] = np.log(data['rkna'] / data['emp']) 
data['a'] = 1 - data['labsh'] 

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['x'] = (grouped_data['k_pc'].diff() * 100) * (data['a']) 

grouped_data = data.groupby('countrycode')

data['y'] = data['g']  - data['x']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean','a': "mean",'labsh': 'mean','x': 'mean','y': 'mean'})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']
summary['TFP Share'] = summary['labsh']
summary['Capital Deepening'] = summary['x']
summary['TFP growth'] = summary['y']

# Print output
#print(summary['Growth Rate'],summary['Capital Share'],summary['TFP Share'],summary['Capital Deepening'],summary['TFP growth'])
print(summary)

                    g         a     labsh         x         y  Growth Rate  \
countrycode                                                                  
CAN          0.887700  0.346190  0.653810  0.634013  0.253687     0.887700   
DEU          0.661055  0.370730  0.629270  0.437718  0.223337     0.661055   
FRA          0.829236  0.381194  0.618806  0.568817  0.260418     0.829236   
GBR          1.015544  0.412993  0.587007  0.573001  0.442543     1.015544   
ITA         -0.037181  0.484835  0.515165  0.533835 -0.571016    -0.037181   
JPN          0.710956  0.430601  0.569399  0.631053  0.079903     0.710956   
USA          1.536985  0.393260  0.606740  0.780325  0.756660     1.536985   

             Capital Share  TFP Share  Capital Deepening  TFP growth  
countrycode                                                           
CAN               0.346190   0.653810           0.634013    0.253687  
DEU               0.370730   0.629270           0.437718    0.223337  
FRA          